In [0]:
#Installing the library
pip install keras-efficientnets

In [0]:
import keras_efficientnets

In [3]:
print(keras_efficientnets.__version__)

'0.1.7'

In [0]:
import tensorflow as tf
import keras

#Mounting the drive(since working on Google Colab)
from google.colab import drive
drive.mount('/content/drive')
# Change the path to the dataset folder
%cd /content/drive/My\ Drive

import os

import numpy

from keras_efficientnets import EfficientNetB0

from keras.preprocessing import image

from keras.models import Model

from keras.layers import Dense, GlobalAveragePooling2D

from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt

%matplotlib inline

from keras.layers import Dense

from keras.layers import Dropout


In [0]:
train_datagen= ImageDataGenerator()
valid_datagen= ImageDataGenerator()
test_datagen= ImageDataGenerator()

#Collecting the train dataset and coverting it to the size of (224,224)
train_generator = train_datagen.flow_from_directory(

    directory='PATH/TO/TRAIN/DATASET' ,

    target_size=(224, 224),

    color_mode="rgb",

    batch_size=32,

    class_mode="categorical",

    shuffle=True,

    seed=42)

#Collecting the validation dataset and coverting it to the size of (224,224)
valid_generator = valid_datagen.flow_from_directory(

    directory='PATH/TO/VALID/DATASET',

    target_size=(224, 224),

    color_mode="rgb",

    batch_size=32,

    class_mode="categorical",

    shuffle=True,

    seed=42)

#Test dataset for the final evaluation of model
test_generator= test_datagen.flow_from_directory(

   directory="PATH/TO/TEST/DATASET",

   target_size=(224,224),color_mode="rgb",

   class_mode="categorical",shuffle=False,seed=42)



STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size

STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

STEP_SIZE_TEST = test_generator.n//test_generator.batch_size

In [0]:
base_model = EfficientNetB0(input_shape=[224,224,3], classes=7, include_top=False, weights='imagenet')

x = base_model.output

x = GlobalAveragePooling2D(name='avg_pool')(x)

x = Dense(1024, activation='relu')(x)

x= Dropout(0.5)(x)

predictions = Dense(7, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [0]:
#unfreezing the layers form 'multiply_16' till last layer
set_trainable = False
for layer in base_model.layers:
    if layer.name == 'multiply_16':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [0]:
#unfreezing all the layers
#Use only not using the above method

#for layer in base_model.layers:
#    layer.trainable = True

In [0]:
model.compile(optimizer='adam',

              loss='categorical_crossentropy',

              metrics=['accuracy'])

In [0]:
print(model.summary())

In [0]:
#Training of the model
cb = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

model.fit_generator(generator=train_generator,validation_data=valid_generator,validation_steps=STEP_SIZE_VALID,steps_per_epoch=STEP_SIZE_TRAIN,epochs=15,callbacks = [cb])

In [0]:
#Saving the model
model.save("PATH/TO/SAVE/MODEL/EfficientB0.model")

##Fine-tuning the model
### *If required*

In [0]:
from efficientnet import load_model
model = load_model("PATH/TO/SAVED/MODEL/EfficientB0.model")
set_trainable = False
for layer in model.layers:
    if layer.name == 'multiply_16':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model.compile(optimizer='adam',

              loss='categorical_crossentropy',

              metrics=['accuracy'])

print(model.summary())

model.fit_generator(generator=train_generator,validation_data=valid_generator,validation_steps=STEP_SIZE_VALID,steps_per_epoch=STEP_SIZE_TRAIN,epochs=9)


In [0]:
# Evalutating the model on the test dataset
model.evaluate_generator(test_generator,steps = STEP_SIZE_TEST, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

In [0]:
# Conversion of the model into Tflite form
# It's a necessay step as tflite form will be deployed on raspberry pi

import tensorflow as tf
import numpy as np
converter  = tf.lite.TFLiteConverter.from_keras_model_file("/content/drive/My Drive/Efficient_nets/EfficientB0_new.model", input_shapes = {'input_1' : [1,224,224,3]})

tflite_model  = converter.convert()
#Saving the model in tflite form
open("/content/drive/My Drive/Efficient_nets/EfficientB0_new.tflite", "wb").write(tflite_model)

## F-1 Score of the model


In [0]:
validation_data_dir = "PATH/TO/DATASET"
import glob
allvalidimgpaths=glob.glob(os.path.join(validation_data_dir, '*/*.jpeg'))
allvalidimgpaths+=glob.glob(os.path.join(validation_data_dir, '*/*.jpg'))
import random
random.shuffle(allvalidimgpaths)

In [0]:
from keras.preprocessing.image import load_img, img_to_array
validation_imgs = [img_to_array(load_img(img, target_size=(224,224))) for img in allvalidimgpaths]

In [0]:
validation_labels= [fn.split('/')[-2] for fn in allvalidimgpaths]

import numpy as np
np.array(validation_imgs)

print(np.shape(validation_imgs))

In [0]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
#LABELS
le.fit(["black eyed beans","chana dal","kidney beans","masoor dal","moong dal","toor dal","urad dal"])
validation_labels_enc = le.transform(validation_labels)

In [0]:
print(validation_labels[5:9])
print(validation_labels_enc[5:9])

In [0]:
newvalids=validation_imgs[:]

valid_imgs_array=numpy.asarray(validation_imgs)

print(valid_imgs_array.shape)
valid_imgs_array.astype('float32')

valid_imgs_array/=255

test_predictions=model.predict(valid_imgs_array)


In [0]:
import numpy as np
newpreds= np.argmax(test_predictions,axis=1)

from sklearn.metrics import classification_report

#Printing the F1 scores
print(classification_report(validation_labels_enc, newpreds, target_names=["black eyed beans","chana dal","kidney beans","masoor dal","moong dal","toor dal","urad dal"]))